In [1]:
import os
import io
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
import os
os.environ['HF_HUB_TIMEOUT'] = '600'  # Timeout in seconds
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [3]:
df=pd.read_csv('df_tot.csv')

In [5]:
def text_preprocessing(text):
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [14]:
MAX_LEN = 128
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (list): List of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for t in data:
        # Ensure the input is a string, convert non-strings to empty string
        if not isinstance(t, str):
            t = str(t) if not pd.isna(t) else ""

        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(t),  # Preprocess sentence
            add_special_tokens=True,     # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,          # Max length to truncate/pad
            truncation=True,             # Activate truncation
            padding='max_length',        # Pad sentence to max length
            return_attention_mask=True   # Return attention mask
        )

        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks


In [11]:
tokenizer = AutoTokenizer.from_pretrained('tunibert')

model = AutoModelForSequenceClassification.from_pretrained('tunibert')

In [26]:
model3b=BertModel.from_pretrained('bert-base-uncased')

c:\Users\moham\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\moham\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [28]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier_3(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier_3, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate BERT model
        self.bert = model3b

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)

        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: total: 0 ns
Wall time: 0 ns


In [30]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model_3(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier_3(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [31]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    #bert_classifier = model(freeze_bert)
    bert_classifier=model
    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [15]:
import random
import time
import torch as nn
import torch
import torch.nn as nn

# Specify loss function
loss_fn = nn.CrossEntropyLoss()


def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)


In [16]:
import pandas as pd

df_positive = df[df['label'] == 1].sample(n=15000, random_state=42)
df_negative = df[df['label'] == 2].sample(n=15000, random_state=42)

# Concatenate the sampled data
df_downsampled = pd.concat([df_positive, df_negative])

# Shuffle the data to mix the classes
df1 = df_downsampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Verify the distribution
print(df1['label'].value_counts(normalize=True))


label
1    0.5
2    0.5
Name: proportion, dtype: float64


In [17]:
from sklearn.model_selection import train_test_split

# Assume you have your data in X and y
X = df1.text.values
y = df1.label.values

# First, split the data into 80% train and 20% temporary set (which will later be split into validation and test sets)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Now split the temporary set into 50% validation and 50% test (which gives 10% of the original data for each)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")


Training set size: 24000
Validation set size: 3000
Test set size: 3000


In [18]:
print('Tokenizing data at the moment...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

Tokenizing data at the moment...


In [19]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [32]:
set_seed(42)
bert_classifier1, optimizer1, scheduler1 = initialize_model(epochs=3)
bert_classifier2, optimizer2, scheduler2 = initialize_model(epochs=3)
bert_classifier3, optimizer3, scheduler3 = initialize_model_3(epochs=3)

c:\Users\moham\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [78]:
# Load the saved model
model_path2 = 'bert_classifier2.pth'
model_path1 = 'bert_posneg_reqneu.pth'
model_path3 = 'bert_classifier_req_or_neu.pth'

# Assuming you have already defined or loaded `bert_classifier`
# Load the state dict into your BERT classifier
bert_classifier1.load_state_dict(torch.load(model_path1, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
bert_classifier1.eval()  # Set the model to evaluation mode



BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [79]:
bert_classifier2.load_state_dict(torch.load(model_path2, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
bert_classifier2.eval()  # Set the model to evaluation mode



BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [80]:
bert_classifier3.load_state_dict(torch.load(model_path3, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
bert_classifier3.eval()  # Set the model to evaluation mode

BertClassifier_3(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [83]:
import torch

def predict_comment(comment, model, tokenizer, max_len=MAX_LEN):
    """Preprocess the input comment and use the model to predict the label.

    Args:
    - comment (str): The input comment to classify.
    - model (torch.nn.Module): The trained BERT model.
    - tokenizer (transformers.PreTrainedTokenizer): The tokenizer ass   ociated with the BERT model.
    - max_len (int): Maximum length for padding/truncation.

    Returns:
    - label (int): Predicted label for the comment (1 for positive, 2 for negative).
    """
    # Preprocess the comment
    encoded_comment = tokenizer.encode_plus(
        text_preprocessing(comment),  # Preprocess the comment
        add_special_tokens=True,      # Add special tokens [CLS] and [SEP]
        max_length=max_len,           # Pad or truncate to max length
        padding='max_length',         # Pad to max length
        truncation=True,              # Truncate longer sentences
        return_attention_mask=True,   # Generate attention mask
        return_tensors='pt'           # Return PyTorch tensors
    )

    # Move tensors to the device (GPU/CPU)
    input_ids = encoded_comment['input_ids'].to(device)
    attention_mask = encoded_comment['attention_mask'].to(device)

    # Put the model in evaluation mode
    model.eval()

    with torch.no_grad():
        # Perform forward pass and get logits
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Convert logits to probabilities
    probabilities = torch.softmax(logits, dim=1)

    # Get the predicted label (0, 1, or 2)
    predicted_label = torch.argmax(probabilities, dim=1).item()

    if predicted_label == 0:  # If the prediction is neutral
        # Set predicted_label to the index of the second highest probability
        probabilities[0, predicted_label] = -float('inf')  # Mask the neutral probability
        predicted_label = torch.argmax(probabilities, dim=1).item()

    return predicted_label

import torch
import torch.nn.functional as F

def predict_comment_posneg_reqneu(comment, model, tokenizer, max_len=MAX_LEN):
    """Preprocess the input comment and use the model to predict the binary label.

    Args:
    - comment (str): The input comment to classify.
    - model (torch.nn.Module): The trained BERT model.
    - tokenizer (transformers.PreTrainedTokenizer): The tokenizer associated with the BERT model.
    - max_len (int): Maximum length for padding/truncation.

    Returns:
    - label (int): Predicted label for the comment (0 for neutral/request, 1 for positive/negative).
    """
    # Preprocess the comment
    encoded_comment = tokenizer.encode_plus(
        comment,                       # Comment to preprocess
        add_special_tokens=True,      # Add special tokens [CLS] and [SEP]
        max_length=max_len,           # Pad or truncate to max length
        padding='max_length',         # Pad to max length
        truncation=True,              # Truncate longer sentences
        return_attention_mask=True,   # Generate attention mask
        return_tensors='pt'           # Return PyTorch tensors
    )

    # Move tensors to the device (GPU/CPU)
    input_ids = encoded_comment['input_ids'].to(device)
    attention_mask = encoded_comment['attention_mask'].to(device)

    # Put the model in evaluation mode
    model.eval()

    with torch.no_grad():
        # Perform forward pass and get logits
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Apply softmax to logits for multi-class classification
    probs = F.softmax(logits, dim=1)  # Convert logits to probabilities

    # Get the predicted class
    predicted_class = torch.argmax(probs, dim=1).item()

    # Map the 3-class output to binary labels
    if predicted_class == 0:
        predicted_label = 0  # Neutral/Request
    elif predicted_class == 1:
        predicted_label = 1  # Positive/Negative
    elif predicted_class == 2:
        predicted_label = 1  # Assuming class 2 is also Positive/Negative
    else:
        raise ValueError("Unexpected predicted class.")

    return predicted_label

import torch
import numpy as np

import torch
import torch.nn.functional as F

def predict_single_input(text, model, tokenizer, max_len=128):
    # Set the device to GPU if available, otherwise use CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Move model to the correct device
    model.to(device)

    # Tokenize the input
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Set the model to evaluation mode
    model.eval()

    # Predict the class probabilities
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        probs = torch.nn.functional.softmax(outputs, dim=1)

    # Get the predicted class (0 or 1)
    pred_class = np.argmax(probs.cpu().numpy(), axis=1)[0]

    # Map the predicted class to the corresponding label
    label = 'neutral' if pred_class == 0 else 'request'

    return label




In [92]:
# Example message to classify
input_text = "kifech nsob f puce te3i"

# Predict the label using the `predict_comment_posneg_reqneu` function
predicted_label = predict_comment_posneg_reqneu(input_text, bert_classifier1, tokenizer)

# Output the result
print(f"Predicted label: {predicted_label}")

Predicted label: 0


In [120]:
# Example message to classify
input_text = "Raj3oulna YALLAJAW mta3 600 mo yehdikom"

# Predict the label using the `predict_comment_posneg_reqneu` function
predicted_label = predict_single_input(input_text, bert_classifier3, tokenizer)

# Output the result
print(f"The input text is classified as: {predicted_label}")


IndexError: index out of range in self

In [112]:
# Example message to classify
input_text = "ana manich fehmek"

# Step 1: Use predict_comment_posneg_reqneu to classify as 0 (neutral/request) or 1 (positive/negative)
predicted_label = predict_comment_posneg_reqneu(input_text, bert_classifier1, tokenizer)

# Step 2: Based on the output of the first model, call the appropriate function
if predicted_label == 0:
    # If the label is 0 (neutral/request), use predict_comment with bert_classifier2
    final_predicted_label = predict_single_input(input_text, bert_classifier3, tokenizer)
    
    if final_predicted_label == "request":
        print("Final predicted label: REQUEST")
    elif final_predicted_label == "neutral":
        print("Final predicted label: NEUTRAL")
    
elif predicted_label == 1:
    # If the label is 1 (positive/negative), use predict_comment with bert_classifier3
    final_predicted_label = predict_comment(input_text, bert_classifier2, tokenizer)
    
    if final_predicted_label == 2:
        print("Final predicted label: Negative")
    elif final_predicted_label == 1:
        print("Final predicted label: positive")


Final predicted label: positive
